In [ ]:
import sys
print(sys.executable)

In [ ]:
import tensorflow as tf
print("GPUs detectadas:", tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from pathlib import Path
import os
import random
from plyfile import PlyData, PlyElement
import open3d as o3d
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
import tensorflow as tf
print("Versión de cuDNN:", tf.sysconfig.get_build_info()["cudnn_version"])
print("Versión de CUDA:", tf.sysconfig.get_build_info()["cuda_version"])

In [ ]:
import tensorflow as tf

print("Versión de TensorFlow:", tf.__version__)
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))

# GOOSE Dataset

In [ ]:
import numpy as np
from pathlib import Path
from typing import List, Tuple
from tqdm import tqdm

NUM_POINTS = 4096  # Tamaño de subnube fijo

# Mapeo de categorías
category_mapping = {
    0: [43, 38, 58, 29, 41, 42, 44, 39, 55],  # Construction
    1: [4, 45, 6, 40, 60, 61, 33, 32, 14],  # Object
    2: [7, 22, 9, 26, 11, 21],  # Road
    3: [48, 47, 1, 19, 46, 10, 25],  # Sign
    4: [23, 3, 24, 31, 2],  # Terrain  
    5: [51, 50, 5, 18],  # Drivable Vegetation
    6: [28, 27, 62, 52, 16, 30, 59, 17],  # Non Drivable Vegetation
    7: [13, 15, 12, 36, 57, 49, 20, 35, 37, 34, 63],  # Vehicle
    8: [8, 56, 0, 53, 54],  # Void
}

label_to_category = {label: cat for cat, labels in category_mapping.items() for label in labels}

def map_labels(labels: np.ndarray) -> np.ndarray:
    """Convierte etiquetas en categorías"""
    return np.array([label_to_category.get(label, 8) for label in labels], dtype=np.uint8)

def load_bin_file(bin_path: str, num_points: int = NUM_POINTS, radius: float = 25.0) -> List[Tuple[np.ndarray, np.ndarray]]:
    """
    Carga una nube de puntos y la divide en subnubes de tamaño fijo `num_points`.

    Retorna:
        Lista de tuplas (subnube, índices) para cada subnube generada.
    """
    # Cargar la nube de puntos
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3]

    # Filtrar puntos dentro del radio dado
    distances = np.linalg.norm(points, axis=1)
    mask = distances <= radius
    indices = np.arange(len(points))[mask]
    points = points[mask]

    num_available = points.shape[0]

    if num_available < num_points:
        return []  # Si no hay suficientes puntos, se descarta la nube

    # Ajustar el número de puntos a un múltiplo exacto de `num_points`
    num_valid = num_available - (num_available % num_points)

    # Seleccionar los primeros `num_valid` puntos
    points = points[:num_valid]
    indices = indices[:num_valid]

    # Dividir en subnubes de `num_points`
    num_subnubes = num_valid // num_points
    subnubes = [
        (points[i * num_points: (i + 1) * num_points], indices[i * num_points: (i + 1) * num_points])
        for i in range(num_subnubes)
    ]

    return subnubes

def load_label_file(label_path: str, indices: np.ndarray) -> np.ndarray:
    """Carga las etiquetas y las mapea a las categorías correspondientes."""
    labels = np.fromfile(label_path, dtype=np.uint32) & 0xFFFF
    return map_labels(labels[indices])

def load_dataset(bin_files: List[str], label_files: List[str], num_points: int = NUM_POINTS) -> Tuple[np.ndarray, np.ndarray]:
    """
    Carga el conjunto de datos dividiendo las nubes grandes en subnubes de tamaño `num_points`.
    """
    x_data, y_data = [], []

    for bin_f, label_f in tqdm(zip(bin_files, label_files), total=len(bin_files), desc="Cargando datos"):
        subnubes = load_bin_file(bin_f, num_points)

        for points, indices in subnubes:
            labels = load_label_file(label_f, indices)
            x_data.append(points)
            y_data.append(labels)

    return np.array(x_data, dtype=np.float32), np.array(y_data, dtype=np.uint8)

def get_file_paths(data_dir: str) -> List[str]:
    """Obtiene rutas de archivos en un directorio."""
    return sorted([str(f) for f in Path(data_dir).glob("*.*")])

def load_all_data(x_train_dir: str, y_train_dir: str, x_val_dir: str, y_val_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Carga los datos de entrenamiento y validación en subnubes de tamaño fijo."""
    x_train_files = get_file_paths(x_train_dir)
    y_train_files = get_file_paths(y_train_dir)
    x_val_files = get_file_paths(x_val_dir)
    y_val_files = get_file_paths(y_val_dir)

    assert len(x_train_files) == len(y_train_files), "Número de archivos x_train y y_train no coincide."
    assert len(x_val_files) == len(y_val_files), "Número de archivos x_val y y_val no coincide."

    print("Cargando datos de entrenamiento...")
    x_train, y_train = load_dataset(x_train_files, y_train_files)

    print("Cargando datos de validación...")
    x_val, y_val = load_dataset(x_val_files, y_val_files)

    return x_train, y_train, x_val, y_val

In [ ]:
x_train_path = "/home/fmartinez/datasets/lidar/train"
y_train_path = "/home/fmartinez/datasets/labels/train"
x_val_path = "/home/fmartinez/datasets_val/lidar/val"
y_val_path = "/home/fmartinez/datasets_val/labels/val"

x_train, y_train, x_val, y_val = load_all_data(x_train_path, y_train_path, x_val_path, y_val_path)

In [ ]:
indices_permutados_train = np.random.permutation(x_train.shape[0])
indices_permutados_val = np.random.permutation(x_val.shape[0])

x_train_shuffle = x_train[indices_permutados_train]
y_train_shuffle = y_train[indices_permutados_train]
x_val_shuffle = x_val[indices_permutados_val]
y_val_shuffle = y_val[indices_permutados_val]

x_train_shuffle.shape, y_train_shuffle.shape, x_val_shuffle.shape, y_val_shuffle.shape

In [ ]:
assert len(x_train) == len(y_train) and len(x_val) == len(y_val)
print(f"El conjunto de entrenamiento tiene {len(y_train)} nubes de puntos de {y_train.shape[1]} puntos")
print(f"El conjunto de entrenamiento tiene {len(y_val)} nubes de puntos de {y_val.shape[1]} puntos")

In [ ]:
import plotly.graph_objects as go
import numpy as np

def plot_3D(xyz, labels):
    """
    Visualiza una nube de puntos en 3D con Plotly.
    - xyz: (num_points, 3) array con coordenadas (X, Y, Z).
    - labels: (num_points,) array con etiquetas semánticas.
    """

    # Definir 9 colores predefinidos en formato RGB
    predefined_colors = [
        "rgb(255, 0, 0)",    # Rojo
        "rgb(0, 255, 0)",    # Verde
        "rgb(0, 0, 255)",    # Azul
        "rgb(255, 255, 0)",  # Amarillo
        "rgb(255, 165, 0)",  # Naranja
        "rgb(128, 0, 128)",  # Púrpura
        "rgb(0, 255, 255)",  # Cian
        "rgb(255, 192, 203)",# Rosa
        "rgb(128, 128, 128)" # Gris
    ]

    # Asignar colores según la etiqueta (se asume que las etiquetas van de 0 a 8)
    point_colors = [predefined_colors[label % len(predefined_colors)] for label in labels]

    # Crear la figura 3D en Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=xyz[:, 0], y=xyz[:, 1], z=xyz[:, 2],  # Coordenadas X, Y, Z
        mode='markers',
        marker=dict(size=1, color=point_colors, opacity=0.8)  # Color basado en etiquetas
    ))

    # Configurar etiquetas y título
    fig.update_layout(
        title="Nube de Puntos con Etiquetas Semánticas",
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
    )

    fig.show()

In [ ]:
plot_3D(x_train_shuffle[500], y_train_shuffle[500])

In [ ]:
import numpy as np
import plotly.express as px

unique_classes, class_counts = np.unique(y_train, return_counts=True)

print(len(y_train))
for cls, count in zip(unique_classes, class_counts):
    print(f"Clase {cls}: {count} puntos")

fig = px.bar(x=unique_classes, y=class_counts, labels={'x': 'Clase', 'y': 'Cantidad de puntos'},
             title='Distribución de etiquetas en y_train')
fig.show()

In [ ]:
# Calcular pesos inversamente proporcionales a la frecuencia de cada clase
total_samples = len(y_train.flatten())  # Total de puntos
class_weights = {cls: total_samples / (len(unique_classes) * count) for cls, count in zip(unique_classes, class_counts)}

print("Pesos de las clases:", class_weights)

len(unique_classes)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

MAX_POINTS = 4096 # 256, 512, 1024, 2048, 4096, 8192, 16384, 32768

def tnet(inputs, num_features):
    x = layers.Conv1D(64, 1, activation='relu', padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(1024, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(num_features * num_features, kernel_initializer='zeros',
                     bias_initializer=tf.keras.initializers.Constant(tf.eye(num_features).numpy().flatten()))(x)
    transform_matrix = layers.Reshape((num_features, num_features))(x)

    def transform(inputs_and_matrix):
        inputs, matrix = inputs_and_matrix
        return tf.matmul(inputs, matrix)

    transformed_inputs = layers.Lambda(transform)([inputs, transform_matrix])
    transformed_inputs = layers.Lambda(lambda t: tf.ensure_shape(t, (None, MAX_POINTS, num_features)))(transformed_inputs)

    return transformed_inputs

def build_pointnet(num_classes, input_dim=3, max_points=MAX_POINTS):
    inputs = tf.keras.Input(shape=(None, input_dim))

    x = tnet(inputs, input_dim)

    x = layers.Conv1D(64, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = tnet(x, 64)

    x = layers.Conv1D(1024, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    global_features = layers.GlobalMaxPooling1D()(x)
    global_features = layers.Lambda(lambda t: tf.expand_dims(t, axis=1))(global_features)
    global_features = layers.Lambda(lambda t: tf.repeat(t, repeats=max_points, axis=1))(global_features)

    x = layers.Lambda(lambda t: tf.ensure_shape(t, (None, max_points, 1024)))(x)

    x = layers.concatenate([x, global_features], axis=-1)

    x = layers.Conv1D(512, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(256, 1, activation='relu', padding="same")(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Conv1D(num_classes, 1, activation='softmax')(x)

    return Model(inputs, outputs)

In [ ]:
import tensorflow as tf

class MeanIoUWrapper(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="mean_iou_wrapper", **kwargs):
        super(MeanIoUWrapper, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_labels = tf.argmax(y_pred, axis=-1)  # Convertir (batch, 16384, 9) -> (batch, 16384)
        self.metric.update_state(y_true, y_pred_labels)

    def result(self):
        return self.metric.result()

    def reset_state(self):
        self.metric.reset_state()

class MeanIoUWrapper_2(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name="mean_iou_wrapper", **kwargs):
        super(MeanIoUWrapper, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.int32)  
        y_pred_labels = tf.argmax(y_pred, axis=-1)  

        self.metric.update_state(y_true, y_pred_labels, sample_weight)  

    def result(self):
        return self.metric.result()

    def reset_state(self):
        self.metric.reset_state()


In [ ]:
import tensorflow as tf

# Pesos ajustados
adjusted_weights = {
    0: 0.80,  # Clase frecuente
    1: 1.0,   # Clases minoritarias quedan en 1
    2: 1.0,
    3: 1.0,
    4: 0.65,   # Clase frecuente
    5: 0.55,  # Clase frecuente
    6: 0.3,   # Clase muy frecuente
    7: 1.0,
    8: 1.0
}

# Convertir a vector NumPy
alpha_vector_adjusted = np.array([adjusted_weights[i] for i in range(len(adjusted_weights))], dtype=np.float32)

# Convertir a tensor de TensorFlow
alpha_tensor_adjusted = tf.constant(alpha_vector_adjusted, dtype=tf.float32)

def weighted_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int32)
    sample_weights = tf.gather(class_weight_tensor, y_true)  # Asigna el peso según la clase
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss * sample_weights  # Escala la pérdida por el peso de la clase

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np

class PointNetSegLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=None, gamma=0, dice=False):
        """
        Implementación de Focal Loss + Dice Loss para segmentación en TensorFlow.
        
        Args:
            alpha: Tensor con los pesos de clase (influencia de cada clase en la pérdida).
            gamma: Factor de Focal Loss. Si gamma > 0, enfatiza ejemplos difíciles.
            dice: Si es True, se suma la Dice Loss.
        """
        super(PointNetSegLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.dice = dice

    def call(self, y_true, y_pred):
        """
        Calcula la pérdida combinada (Focal Loss + Dice Loss si está activada).
        
        Args:
            y_true: Tensor con etiquetas verdaderas (batch_size, num_points).
            y_pred: Tensor con logits o probabilidades predichas (batch_size, num_points, num_classes).
            
        Returns:
            Pérdida escalada con los pesos de clase y Focal Loss.
        """
        num_classes = tf.shape(y_pred)[-1]

        # Convertir etiquetas a one-hot
        y_true = tf.cast(y_true, tf.int32)  # Asegurar tipo int32
        y_true_one_hot = tf.one_hot(y_true, depth=num_classes)

        # Cross Entropy Loss base
        ce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction="none")(y_true_one_hot, y_pred)

        # Aplicar pesos de clase (alpha) si están definidos
        if self.alpha is not None:
            y_true_flat = tf.reshape(y_true, [-1])  # Convertir a 1D para evitar errores
            sample_weights = tf.gather(self.alpha, y_true_flat)  # Extraer pesos

            # Ajustar la forma de `sample_weights` para que coincida con `ce_loss`
            sample_weights = tf.reshape(sample_weights, tf.shape(ce_loss))

            # Escalar la pérdida
            ce_loss *= sample_weights

        # Aplicar Focal Loss si gamma > 0
        if self.gamma > 0:
            y_pred_softmax = tf.nn.softmax(y_pred, axis=-1)  # Convertir logits a probabilidades
            pt = tf.reduce_sum(y_true_one_hot * y_pred_softmax, axis=-1)  # Probabilidad del target correcto
            focal_loss = (1 - pt) ** self.gamma * ce_loss  # Focal Loss
        else:
            focal_loss = ce_loss

        # Promediar la pérdida
        loss = tf.reduce_mean(focal_loss)

        # Si Dice Loss está activada, se suma a la pérdida total
        if self.dice:
            dice_loss = self.compute_dice_loss(y_true_one_hot, y_pred_softmax)
            loss += dice_loss

        return loss

    @staticmethod
    def compute_dice_loss(y_true, y_pred, eps=1e-6):
        """
        Calcula la Dice Loss, que mide la superposición entre predicciones y etiquetas.
        """
        intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])  # Suma sobre puntos y clases
        union = tf.reduce_sum(y_true + y_pred, axis=[1, 2])

        dice = (2. * intersection + eps) / (union + eps)
        return 1 - tf.reduce_mean(dice)

# Crear la función de pérdida con los pesos ajustados
loss_fn = PointNetSegLoss(alpha=alpha_tensor_adjusted, gamma=0.5, dice=True)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class PointCloudSequence(Sequence):
    def __init__(self, x_data, y_data, batch_size=32, shuffle=True, augment_prob=0.25):
        """
        DataLoader con normalización y aumentaciones para segmentación de nubes de puntos.

        Args:
            x_data: numpy array con shape (m, 4096, 3) (nubes de puntos).
            y_data: numpy array con shape (m, 4096) (etiquetas por punto).
            batch_size: Número de nubes por batch.
            shuffle: Si True, reorganiza los datos en cada epoch.
            augment_prob: Probabilidad de aplicar aumentaciones (0.25 = 25%).
        """
        self.x_data = x_data  # Nubes de puntos
        self.y_data = y_data  # Etiquetas correspondientes
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment_prob = augment_prob
        self.indices = np.arange(len(self.x_data))
        self.on_epoch_end()

    def __len__(self):
        """Número total de batches por epoch."""
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        """Genera un batch de datos con normalización y aumentaciones."""
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        x_batch = np.array([self.process_cloud(self.x_data[idx]) for idx in batch_indices])
        y_batch = np.array([self.y_data[idx] for idx in batch_indices])  # Etiquetas sin modificar

        return x_batch, y_batch  # Devuelve (inputs, etiquetas)

    def process_cloud(self, cloud):
        """Normaliza y aplica aumentaciones con probabilidad augment_prob."""
        cloud = self.normalize_cloud(cloud)

        # Aplicar aumentación con 25% de probabilidad
        if np.random.rand() < self.augment_prob:
            cloud = self.apply_augmentations(cloud)

        return cloud

    def normalize_cloud(self, cloud):
        """Normaliza una nube de puntos individualmente."""
        return (cloud - np.mean(cloud, axis=0)) / (np.std(cloud, axis=0) + 1e-6)

    def apply_augmentations(self, cloud):
        """Aplica ruido gaussiano y rotaciones aleatorias."""
        cloud += np.random.normal(0.0, 0.01, cloud.shape)  # Añadir ruido gaussiano

        # Rotación aleatoria en ejes X, Y, Z
        cloud = np.dot(cloud, self.random_rotation_matrix())

        return cloud

    def random_rotation_matrix(self):
        """Genera una matriz de rotación aleatoria en 3D."""
        theta = np.random.uniform(-np.pi, np.pi)
        phi = np.random.uniform(-np.pi, np.pi)
        psi = np.random.uniform(-np.pi, np.pi)

        rot_x = np.array([
            [1, 0, 0],
            [0, np.cos(theta), -np.sin(theta)],
            [0, np.sin(theta), np.cos(theta)]
        ])

        rot_y = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])

        rot_z = np.array([
            [np.cos(psi), -np.sin(psi), 0],
            [np.sin(psi), np.cos(psi), 0],
            [0, 0, 1]
        ])

        return rot_x @ rot_y @ rot_z  # Producto matricial para obtener la rotación final

    def on_epoch_end(self):
        """Baraja los datos al final de cada epoch si shuffle=True."""
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
# Crear DataLoaders
train_loader = PointCloudSequence(x_train, y_train, batch_size=32, shuffle=True)
val_loader = PointCloudSequence(x_val, y_val, batch_size=32, shuffle=False, augment_prob=0.0)

# Verificar un batch de datos
x_sample, y_sample = train_loader[0]  # Primer batch
print("Shape de x_batch:", x_sample.shape)  # Debe ser (8, 4096, 3)
print("Shape de y_batch:", y_sample.shape)  # Debe ser (8, 4096)

In [ ]:
INPUT_DIM = 3
NUM_CLASSES = 9

# Definir el decay lineal con PolynomialDecay
initial_lr = 0.4  # Learning rate inicial
final_lr = 0.1  # Learning rate final
decay_epochs = 4   # Número de épocas para decaer linealmente

lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=decay_epochs,
    end_learning_rate=final_lr,
    power=1.0,  # Power=1.0 garantiza un decaimiento estrictamente lineal
    cycle=False  # No reiniciar después del decaimiento
)

# Definir el modelo PointNet
model = build_pointnet(num_classes=NUM_CLASSES, input_dim=INPUT_DIM)
mean_iou_wrapper = MeanIoUWrapper(num_classes=NUM_CLASSES)


# Definir el optimizador con el scheduler
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compilar el modelo
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=["accuracy", mean_iou_wrapper]
)

In [ ]:
import gc
import tensorflow as tf

# Liberar memoria de TensorFlow
tf.keras.backend.clear_session()

# Forzar liberación de memoria en Python
gc.collect()

In [ ]:
import tensorflow as tf

history = model.fit(
    train_loader, 
    validation_data=val_loader,
    epochs=4,
    verbose=1,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

In [ ]:
model.save("pointnet_goose_16k_4.keras")

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Supongamos que estos vienen de tu modelo entrenado (Keras history)
loss = history.history['loss']
val_loss = history.history.get('val_loss')
mean_iou = history.history.get('mean_iou_wrapper')
val_mean_iou = history.history.get('val_mean_iou_wrapper')
accuracy = history.history.get('accuracy')
val_accuracy = history.history.get('val_accuracy')

# Crear figura con 1 fila y 3 columnas de subplots
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Loss Curve", "Mean IoU Curve", "Accuracy Curve")
)

# 1) GRÁFICO DE PÉRDIDA (col=1)
epochs_loss = list(range(1, len(loss) + 1))

fig.add_trace(
    go.Scatter(
        x=epochs_loss,
        y=loss,
        mode='lines+markers',
        name='Train Loss'
    ),
    row=1, col=1
)

if val_loss:
    epochs_val_loss = list(range(1, len(val_loss) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_val_loss,
            y=val_loss,
            mode='lines+markers',
            name='Validation Loss'
        ),
        row=1, col=1
    )

fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=1, range=[0, 10])  # Rango ajustado

# 2) GRÁFICO DE Mean IoU (col=2) -> Limitar valores a [0,1]
if mean_iou:
    mean_iou = np.clip(mean_iou, 0, 1)
    epochs_mean_iou = list(range(1, len(mean_iou) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_mean_iou,
            y=mean_iou,
            mode='lines+markers',
            name='Train Mean IoU'
        ),
        row=1, col=2
    )

    if val_mean_iou:
        val_mean_iou = np.clip(val_mean_iou, 0, 1)
        epochs_val_mean_iou = list(range(1, len(val_mean_iou) + 1))
        fig.add_trace(
            go.Scatter(
                x=epochs_val_mean_iou,
                y=val_mean_iou,
                mode='lines+markers',
                name='Validation Mean IoU'
            ),
            row=1, col=2
        )

fig.update_xaxes(title_text='Epochs', row=1, col=2)
fig.update_yaxes(title_text='Mean IoU', row=1, col=2, range=[0, 1])

# 3) GRÁFICO DE ACCURACY (col=3) -> Limitar valores a [0,1]
if accuracy:
    accuracy = np.clip(accuracy, 0, 1)
    epochs_acc = list(range(1, len(accuracy) + 1))
    fig.add_trace(
        go.Scatter(
            x=epochs_acc,
            y=accuracy,
            mode='lines+markers',
            name='Train Accuracy'
        ),
        row=1, col=3
    )

    if val_accuracy:
        val_accuracy = np.clip(val_accuracy, 0, 1)
        epochs_val_acc = list(range(1, len(val_accuracy) + 1))
        fig.add_trace(
            go.Scatter(
                x=epochs_val_acc,
                y=val_accuracy,
                mode='lines+markers',
                name='Validation Accuracy'
            ),
            row=1, col=3
        )

fig.update_xaxes(title_text='Epochs', row=1, col=3)
fig.update_yaxes(title_text='Accuracy', row=1, col=3, range=[0, 1])

# Ajustes generales de la figura
fig.update_layout(
    width=1300,
    height=500,
    showlegend=True
)

fig.show()